In [19]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [20]:
product_data = pd.read_csv('../data/market_integrates_1.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.688920        20.558583  31.569389   
1             1         2         5.382306        20.967530  29.038596   
2             1         3         5.067265        20.420601  33.823156   
3             1         4         5.935766        20.037968  34.395821   
4             1         5         5.167931        20.257820  31.311906   
..          ...       ...              ...              ...        ...   
495         100         1         5.688920        20.558583  31.127981   
496         100         2         5.382306        20.967530  29.259691   
497         100         3         5.067265        20.420601  33.911192   
498         100         4         5.935766        20.037968  32.091197   
499         100         5         5.167931        20.257820  30.186119   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0        21.475068  0.003420  0.0345

In [21]:
# Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
product_data_complete = product_data.rename(columns={'labor':'supply_instrument0', 'capital':'supply_instruments1', 'marginal_cost': 'supply_instrument2'})
print(product_data_complete)



     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.688920        20.558583  31.569389   
1             1         2         5.382306        20.967530  29.038596   
2             1         3         5.067265        20.420601  33.823156   
3             1         4         5.935766        20.037968  34.395821   
4             1         5         5.167931        20.257820  31.311906   
..          ...       ...              ...              ...        ...   
495         100         1         5.688920        20.558583  31.127981   
496         100         2         5.382306        20.967530  29.259691   
497         100         3         5.067265        20.420601  33.911192   
498         100         4         5.935766        20.037968  32.091197   
499         100         5         5.167931        20.257820  30.186119   

     supply_instrument2    shares   profits   markups  e_quantity  \
0             21.475068  0.003420  0.03451

In [22]:
# the endog regressor
X1_formulation = pyblp.Formulation('1 + prices + characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

product_formulations = (X1_formulation, X2_formulation)
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [23]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [24]:
pr_integration = pyblp.Integration('product', size=5)


In [25]:
mc_problem = pyblp.Problem(product_formulations, product_data_complete, integration=mc_integration)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    F    I     K1    K2    MD 
---  ---  ---  ----  ----  ----  ----
100  500   5   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N    F    I     K1    K2    MD 
---  ---  ---  ----  ----  ----  ----
100  500   5   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [26]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [27]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evalua